# towson_u

## Load Libraries

In [2]:
# Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# Reqeusts
import requests
# Other tools
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from datetime import date
import csv
import pandas as pd
import time
import json
import fnmatch
import os
import tabula
from tabula.io import read_pdf


## Functions Using Requests

In [3]:
def requests_get_item(url, item):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    if item == "html":       
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        return(soup)
    
    elif item == "pdf":
        page = requests.get(url, headers = headers, stream=True)
        return(page)
    else:
        print("Valid Item Not Selected")

def download_pdfs(ls_pdf_urls, download_path, file_name):
    counter = 0 
    for pdf_url in ls_pdf_urls:
        counter = counter +1
        g = requests_get_item(pdf_url, "pdf")
        with open(f'{download_path}{file_name}_{counter}.pdf', 'wb') as sav:
            for chunk in g.iter_content(chunk_size=1000000):
                sav.write(chunk)
        print(f"download number: {counter}")
                
                
def convert_pdf_to_csv(pdf_directory, csv_directory):
    directory = fr'{pdf_directory}'
    directory_output = fr'{csv_directory}'
    count = 0
    for file in os.listdir(directory):        
        if file.endswith(".pdf"):
            count = count + 1 
            print(f'{directory}{file}: Conversion {count}')
            tabula.convert_into(f'{directory}{file}', f'{directory_output}{count}.csv', output_format="csv", pages='all')
    
    
    
    

## Functions Using Selenium

In [4]:
# Define important paths
executable_path = "C:/Users/nicho/Documents/Code/university_police_logs/webscrapers/chromedriver.exe"
download_path = f"C:\\Users\\nicho\\Documents\\Code\\university_police_logs\\data\\handmade\\gt\\pdfs\\"

# Define Routes (usually in xpaths)

def browser_instance(executable_path, headless, download_path):
    
    
    chrome_options =  Options()
    if headless:
        chrome_options.add_argument("--headless")   
    chrome_options.add_experimental_option("prefs", {
            "download.default_directory": download_path})    
    driver = webdriver.Chrome(executable_path=executable_path, options = chrome_options)
    return driver  

def signIn(driver, url, signin_field, username_field, password_field):  
    driver.get(url)
    sign_in = driver.find_element_by_xpath(signin_field)
    sign_in.click()
    email_field = driver.find_element_by_xpath(username_field)
    password_field = driver.find_element_by_xpath(password_field)
    email_field.send_keys(USERNAME)
    password_field.send_keys(PASSWORD + Keys.RETURN)


## Useful Selenium Concepts
### Waiting until element is clickable
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, download_all_field)))
button.click()

### Try and Expect
try:
    attempt to do something
except:
    do this if the try doesn't work out
    


## General Functions

In [5]:
def write_json_data(data_out, database_name, file_path):
        #writing data to json
        data_out = data_out
        database_name = database_name
        file = f'{file_path}/{database_name}'
        with open(file, 'w') as outfile:
            json.dump(data_out, outfile)
            
def open_json_data(file_path):    
    with open(file_path) as f:
        data = json.load(f)
    return data

### Get links

In [23]:
#script

page = requests_get_item('https://police.umbc.edu/daily-crime-log/' , 'html')
div = page.find('ul', {"class": "section-menu"})
#print(div)
link_list = []

ls_links = div.find_all('a')
for link in ls_links:
    url = (link["href"])
    link_list.append(url)
#     print(link_list)
#     print('---')


link_list = link_list[1:]

all_links = []
for link in link_list:
    page = requests_get_item(link, "html")
    div = page.find("div", {"class": "entry-content"})
    ls_links_2 = div.find_all('a')
    for month_url in ls_links_2:
        url = month_url['href']
        all_links.append(url)
        
    
print(all_links)

download_path = "../data/handmade/umbc/pdfs/"
download_pdfs(all_links, download_path, "umbc")


['https://police.umbc.edu/files/2021/06/Daily-Log-01_2021.pdf', 'https://police.umbc.edu/files/2021/06/Daily-Log-02_2021.pdf', 'https://police.umbc.edu/files/2021/06/Daily-Log-03_2021.pdf', 'https://police.umbc.edu/files/2021/06/Daily-Log-04_2021.pdf', 'https://police.umbc.edu/files/2021/06/Daily-Log-05_2021.pdf', 'https://police.umbc.edu/files/2021/07/Daily-Log-06_2021-1.pdf', 'https://police.umbc.edu/files/2021/07/Daily-Log-07_2021-2.pdf', 'https://police.umbc.edu/files/2021/09/Daily-Log-08_2021.pdf', 'https://police.umbc.edu/files/2021/10/Daily-Log-09_2021-1.pdf', 'https://police.umbc.edu/files/2020/05/Daily-Log-STRK-01_20.pdf', 'https://police.umbc.edu/files/2021/08/Daily-Log-02_20-1.pdf', 'https://police.umbc.edu/files/2020/05/Daily-Log-STRK-03_20.pdf', 'https://police.umbc.edu/files/2020/05/Daily-Log-STRK-04_20-1.pdf', 'https://police.umbc.edu/files/2020/06/Daily-Log-STRK-05_20.pdf', 'https://police.umbc.edu/files/2020/07/Daily-Log-STRK-06_20.pdf', 'https://police.umbc.edu/files/

## Download files

## Now convert 'em